In [107]:
import numpy as np
import pandas as pd
import nltk
import time
nltk.download('stopwords')

from numpy import random as rd
from scipy.special import gammaln
from nltk.corpus import stopwords
from collections import Counter
from collections import defaultdict
from matplotlib import pyplot as plt

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Sara\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Function definitions

## File related

In [112]:
def find_n_lines(filename, encoding, n_tokens):
    """
    We should train on a subset of the corpus, with 100 0000 tokens. Find how many lines this corresponds to.
    """
    
    line_counter = 0
    token_counter = 0

    with open(filename, encoding = encoding) as f:
        for line in f:

            line_counter += 1
            tokens = line.lower().split()

            for token in tokens:
                token_counter += 1

            if token_counter > n_tokens:
                break

    return line_counter


def count_word_frequencies(filename, encoding, n_lines, ignore_list):
    
    freqs = Counter()
    with open(filename, encoding = encoding) as f:
        for i, line in enumerate(f):
            
            tokens = line.lower().split()
            for token in tokens:
                if token not in ignore_list:
                    freqs[token] += 1
                
            if i == n_lines:
                break
                
    return freqs

## Batch related

In [141]:
def create_integer_vocabulary(word_freqs, max_voc_size):
    
    """ 
    Create vocabulary where common words are matched to integers. 
    """
    
    word_list = []

    if len(word_freqs.most_common()) > max_voc_size:
        vocab = word_freqs.most_common()[0:max_voc_size]

    else:
        vocab = word_freqs.most_common()

    for i in range(len(vocab)):
        word_list.append(vocab[i][0])

    # Get pairs of elements    
    tmp = zip(word_list, range(1,max_voc_size+1))
    # Make pairs into a dictionary
    vocab = dict(tmp)

    # Create default dictionary - returns 0 if an undefined key is called
    vocab2 = defaultdict(int)
    vocab2.update(vocab)
    
    return vocab2

def find_batch_dimensions(batch_size, filename, ENCODING):
    """
    Find the length of the longest line in each batch.
    """
    
    counter = 0      # will end up being the number of lines in the document
    len_lines = []   # will contain maximum length of a line in each batch
    tmp_lines = []
    
    with open(filename, encoding=ENCODING) as f:
        for line in f:
            counter+=1
            tokens = line.lower().split()
            tmp_lines.append(len(tokens))

            if (counter % batch_size == 0):
                len_lines.append(max(tmp_lines))
                tmp_lines = []
                
        #This takes care of the last batch if number of lines is not an exact multiple of batch_size
        if (counter % batch_size != 0): 
            len_lines.append(max(tmp_lines)) # if at end of the file
            
    return counter, len_lines
    

def create_batches(batch_size, vocabulary, filename, ENCODING):
    """
    Splits the file into batches of a specified size, and transforms common words to integers.
    The batches are outputted in a numpy array padded with zeros. Words not in the vocabulary are set to -1.
    """
    
    counter, len_lines = find_batch_dimensions(batch_size, filename, ENCODING)
    
    with open(filename, encoding=ENCODING) as f:
        batches=[]
        batch_counter=0
        line_counter=0

        for line in f:
            #This creates a temporary array each time we start a new batch
            if line_counter % batch_size == 0:
                tmp_array=np.zeros(shape=(batch_size,len_lines[batch_counter])) #fill this temporary array

            tokens = line.lower().split()
            line_as_int = list(map(vocabulary.get, tokens))
            line_as_int = [-1 if x is None else x for x in line_as_int] # set None values to -1

            tmp_array[line_counter % batch_size,0:(len(line_as_int))]=line_as_int

            line_counter+=1 #when we done
            if line_counter % batch_size ==0:
                batches.append(tmp_array)
                batch_counter+=1

        # again this takes care of the final batch if number of lines is not multiple of batch_size
        if line_counter % batch_size != 0:
            tmp_array=tmp_array[0:(line_counter % batch_size),:]
            batches.append(tmp_array)
        
    return(counter, batches)

def get_matrix(filename, encoding, n_tokens, ignore_words):
    # Find how many lines we need to read to get the desired number of tokens
    n_docs = find_n_lines(filename, encoding, n_tokens)

    # Count word frequencies in this subset of the file
    word_frequencies = count_word_frequencies(filename, encoding, n_docs, ignore_words)

    # Create an integer vocabulary. Don't remove any words from the vocabulary.
    voc_size = len(word_frequencies)
    vocabulary = create_integer_vocabulary(word_frequencies, voc_size)

    # Turn the document into batches
    lines, batches = create_batches(batch_size=n_docs, vocabulary = vocabulary, filename = filename, ENCODING = encoding)

    # Save only the first batch - this is what we'll analyse
    matrix = batches[0].astype(int)
    return n_docs, matrix, vocabulary, voc_size

## LDA Related

In [96]:
def initialise_everything(n_docs, n_topics, voc_size, int_matrix):
    """
    Initialise some stuff!
    """

    # Number of times that we observe topic z in document d
    ndz = np.zeros((n_docs, n_topics))

    # Number of times that we observe word w in topic z
    nzw = np.zeros((n_topics, voc_size))

    # Counters for documents and topics
    nd = np.zeros(n_docs)
    nz = np.zeros(n_topics)

    # Create dictionary of topics
    topics = {}

    # iterate over documents 
    for d in range(n_docs):

        # i is the index of the word in the document
        # w is the numerical representation of the word
        for i, w in enumerate(int_matrix[d]):

            # Initialise with a random topic
            z = rd.randint(n_topics)
            topics[(d,i)] = z

            # Increase counters
            ndz[d, z] += 1
            nzw[z, w] += 1

            nd[d] += 1
            nz[z] += 1

    return topics, ndz, nzw, nd, nz


def cond_topic_prob(ndz, nzw, nz, nd, w, d, alpha, beta, n_topics):
    """
    Conditional probability of topics. Is this the same formula as in lecture notes?
    """

    left = (nzw[:,w] + beta) / (nz + beta * voc_size)
    right = (ndz[d,:] + alpha) / (nd[d] + alpha * n_topics)

    p_z = left * right
    p_z /= np.sum(p_z)
    
    return p_z


def log_multinomial_beta(alpha, K=None):

    if K is None:
        # alpha is assumed to be a vector
        return np.sum(gammaln(alpha)) - gammaln(np.sum(alpha))
    else:
        # alpha is assumed to be a scalar
        return K * gammaln(alpha) - gammaln(K*alpha)

# This should decrease as training progresses, show it every few training iterations (?)
def loglikelihood(n_topics, voc_size, beta, nzw, ndz):
    likelihood = 0
    
    for z in range(n_topics):
        likelihood += log_multinomial_beta(nzw[z,:] + beta)
        likelihood -= log_multinomial_beta(beta, voc_size)
        
    for d in range(n_docs):
        likelihood += log_multinomial_beta(ndz[d,:] + alpha)
        likelihood -= log_multinomial_beta(alpha, n_topics)
        
    return likelihood

# 1: Write your own code for doing Gibbs sampling for LDA

In [114]:
# Ignore all stopwords in the text!
ignore_words = stopwords.words('english')
also_ignore = [",", ".", '"', "(", ")", "n't", "-", "'", "'s", "'m", "!", "?", ":", ";", "/"]

for item in also_ignore:
    ignore_words.append(item)

In [142]:
filename = "books.txt" 
encoding = "ISO-8859-1"
n_tokens = 10**5

n_docs, matrix, vocabulary, voc_size = get_matrix(filename, encoding, n_tokens, ignore_words)

In [143]:
beta = 0.1
alpha = 0.1
n_topics = 10

max_iterations = 2

# beta = 0.01
# alpha = 0.01
# n_topics = 50

In [148]:
def LDA_Gibbs_Sampler(matrix, voc_size, n_docs, n_topics, max_iterations, alpha, beta):

    start_time = time.time()
    topics, ndz, nzw, nd, nz = initialise_everything(n_docs, n_topics, voc_size, matrix)

    for i in range(max_iterations):
        for d in range(n_docs):
             for j, w in enumerate(matrix[d]):

                    z = topics[(d, j)]
                    ndz[d, z] -= 1
                    nzw[z, w] -= 1
                    nd[d] -= 1
                    nz[z] -= 1

                    p_z = cond_topic_prob(ndz, nzw, nz, nd, w, d, alpha, beta, n_topics)
                    z = rd.multinomial(1, p_z).argmax()

                    ndz[d,z] += 1
                    nzw[z,w] += 1
                    nd[d] += 1
                    nz[z] += 1
                    topics[(d, j)] = z

        print("Iteration", i)
        print("Likelihood", loglikelihood(n_topics, voc_size, beta, nzw, ndz))

    elapsed_time = time.time() - start_time
    print("Elapsed time: ", elapsed_time)
    
    return nzw

In [145]:
word_topic_prob = LDA_Gibbs_Sampler(matrix, voc_size, n_docs, n_topics, max_iterations, alpha, beta)
show_words_by_topic(word_topic_prob, vocabulary, n_topics)

Iteration 0
Likelihood -2958842.840623099
Iteration 1
Likelihood -2939639.8833902585
95.46932053565979


In [154]:
typical_len = 10

def show_words_by_topic(word_topic_prob, vocabulary, n_topics):
    typical_words = []

    for i in range(n_topics):
        arr = word_topic_prob[i,:]
        typical_ints = arr.argsort()[-typical_len-1:-1][::-1]
        #print(typical_ints)

        for search_int in typical_ints:
            for k, v in vocabulary.items(): 
                if v == search_int:
                    typical_words.append(k)

    # Print the most common words in each topic
    typical_words = np.reshape(typical_words, [n_topics, -1])
    print(typical_words)

Assess model performance in tables over the ten top words for the topics.

# Object oriented in Python - would be less messy to do this

In [7]:
class Person:
    
    def __init__(self, name, age):
        self.name = name
        self.age = age
    
    def greeting(self):
        print("Hello, my name is " + self.name)

In [9]:
p1 = Person("Sara", 25)
p1.age

p1.greeting()

Hello, my name is Sara


# 2: Write your own code for doing Gibbs samling on Bigram LDA

as in H. M. Wallach: Topic modeling: beyond bag-of-words. ICML(2006) 977-984. http://dirichlet.net/pdf/wallach06topic.pdf

For corpus use the Amazon book reviews corpus that you also used in Assignment 1. You may have to use only a subset of the documents. A corpus of 100 000 tokens is sufficients size.

Run this for different hyperparameters. For LDA you can try α=β=0.1 and α=β=0.01. (A cross-validation search for optimal values will probably be too slow.) Run also for different numbers of topics, e.g. K=10 and K=50.

For the bigram model, see to that you use a larger hyperparameter value on the diagonal of the transition matrix over the topics. Since each document in this model has a transition matrix over topics rather than just a probability distribution, the number of topics cannot be as large as for LDA. Try K=5 and K=10.